In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import random
import time

### User Input


In [ ]:
# print("Do you want personalized planning by providing some information like Height weight, etc.?")
print("If YES, Press X = 1, Otherwise X = 0")

x = int(input("X = "))


### Calculating BMR and Nutrients Constraints


In [ ]:
#BMR calculation 
def calculate_bmr(gender, weight, height, age):
    if gender.lower() == 'male':
        return (10 * weight) + (6.25 * height) - (5 * age) + 5
    else:
        return (10 * weight) + (6.25 * height) - (5 * age) - 161

if(x==1):
    
    # user = {
    # 'name': 'Jamiya Alom',
    # 'age': 20,
    # 'gender': 'female',
    # 'height': 190,  # in cm
    # 'weight': 75,   # in kg
    # }

    user = {}

    # user input
    user['name'] = input("Enter your name: ")
    user['age'] = int(input("Enter your age: "))
    user['gender'] = input("Enter your gender (male/female): ").lower()
    user['height'] = float(input("Enter your height in cm: "))
    user['weight'] = float(input("Enter your weight in kg: "))

    bmr1= calculate_bmr(user['gender'], user['weight'], user['height'], user['age'])
    print(f"Calculated BMR: {bmr1:.2f} kcal")

    min_cal= bmr1 * 1.2
    max_cal = bmr1 * 1.9

    # Minimum Calory needed
    bmr= bmr1*1.2

    # Protein Calculation
    Pconst = (bmr*0.20)/4

    # Fat Calculation
    Fconst = (bmr * 0.25)/9

    #Sugar Calculation
    Sconst = (bmr * 0.12)/4

    Sodium_const = 2500

    print(f"Minimum Calory: {bmr:.2f}, min_cal: {min_cal}, max_cal: {max_cal}, min Protein: {Pconst}, max_fat: {Fconst}, max_sugar: {Sconst}, max_sodium:{Sodium_const}")
    
else:
    min_cal = 2200
    max_cal = 3500
    Pconst = 110
    Fconst = 61
    Sconst = 66
    Sodium_const = 2500
    print(f"min_cal: {min_cal}, max_cal: {max_cal}, min Protein: {Pconst}, max_fat: {Fconst}, max_sugar: {Sconst}, max_sodium:{Sodium_const}")

### Applying Linear Programming on small data

In [ ]:
costs = [45, 38, 65, 85, 50, 40, 60, 70, 55, 48, 95, 78, 88, 120, 130]
calories = [350, 400, 390, 550, 300, 280, 500, 450, 330, 360, 600, 410, 420, 580, 590]
proteins = [12, 15, 22, 20, 10, 8, 16, 18, 12, 10, 25, 14, 17, 19, 18]
fats = [4, 6, 5, 10, 3, 2, 7, 6, 4, 3, 12, 6, 5, 11, 15]
sugars = [3, 14, 15, 10, 6, 5, 10, 12, 8, 4, 18, 9, 11, 16, 20]

model = pulp.LpProblem("Daily_Meal_Plan", pulp.LpMinimize)

# Binary variables: xi = 1 if food item i is selected
x = [pulp.LpVariable(f"x{i}", cat='Binary') for i in range(15)]

# Objective: Minimize total cost
model += pulp.lpSum(costs[i] * x[i] for i in range(15)), "Total Cost"
n=15
# Constraints
model += pulp.lpSum(x[i] for i in range(n)) >= 3
model += pulp.lpSum(x[i] for i in range(n)) <= 8
model += pulp.lpSum(calories[i] * x[i] for i in range(n)) >= 700
model += pulp.lpSum(calories[i] * x[i] for i in range(n)) <= 4000
model += pulp.lpSum(proteins[i] * x[i] for i in range(n)) >= Pconst
model += pulp.lpSum(fats[i] * x[i] for i in range(n)) <= Fconst
model += pulp.lpSum(sugars[i] * x[i] for i in range(n)) <= Sconst

# Solve the model
model.solve()

# ------------------------
# Output the results
# ------------------------
print("Status:", pulp.LpStatus[model.status])
print("Selected Food Items:")
for i in range(n):
    if x[i].varValue == 1:
        print(f"  Food {i+1}: Cost={costs[i]}, Cal={calories[i]}, Prot={proteins[i]}, Fat={fats[i]}, Sugar={sugars[i]}")
print(f"\nTotal Cost: {pulp.value(model.objective)} Baht")
print("\n")

### Start Working with real-time dataset

In [ ]:
#Data Loading
data = pd.read_csv("../dataset/7_11_food_choices.csv")
data



In [ ]:
#Extracting the data and convert it to the list
prices     = data['Price(baht)'].tolist()
calories  = data['Calories(kCal)'].tolist()
proteins  = data['Protein(g)'].tolist()
fats      = data['Fat(g)'].tolist()
sugars    = data['Sugar(g)'].tolist()
sodium    = data['Sodium(mg)'].tolist()
items     = data['ITEM'].tolist()

n = len(data)  # Total food items

min_item = 3
max_item = 9

# LINEAR PROGRAMMING


In [ ]:
start_time = time.time()

#LP Model
model = pulp.LpProblem("Daily_Meal_Plan", pulp.LpMinimize)

#xi = 1 if item i is selected
x = [pulp.LpVariable(f"x{i}", cat='Binary') for i in range(n)]

#Objective: Minimizing total cost
model += pulp.lpSum(prices[i] * x[i] for i in range(n)), "Total Cost"

#Constraints
model += pulp.lpSum(x[i] for i in range(n)) >= min_item
model += pulp.lpSum(x[i] for i in range(n)) <= max_item
model += pulp.lpSum(calories[i] * x[i] for i in range(n)) >= min_cal
model += pulp.lpSum(calories[i] * x[i] for i in range(n)) <= max_cal
model += pulp.lpSum(proteins[i] * x[i] for i in range(n)) >= Pconst
model += pulp.lpSum(fats[i] * x[i] for i in range(n)) <= Fconst
model += pulp.lpSum(sugars[i] * x[i] for i in range(n)) <= Sconst
model += pulp.lpSum(sodium[i] * x[i] for i in range(n)) <= Sodium_const

#Solve the model
model.solve()

end_time = time.time()
execution_time = end_time - start_time

#Output 
print("Status:", pulp.LpStatus[model.status])
print("\nSelected Food Items:")
total_cost = 0
total_cal = 0
total_protein = 0
total_fat = 0
total_sugar = 0
total_sodium = 0
selected_item = 0

for i in range(n):
    if x[i].varValue == 1:
        selected_item += 1
        print(f"  {items[i]} | Cost: {prices[i]} Baht | Cal: {calories[i]} | Prot: {proteins[i]} | Fat: {fats[i]} | Sugar: {sugars[i]} | Sodium: {sodium[i]}")
        total_cost += prices[i]
        total_cal += calories[i]
        total_protein += proteins[i]
        total_fat += fats[i]
        total_sugar += sugars[i]
        total_sodium += sodium[i]

print("\nSummary of Best Meal Plan:")
print(f"\nTotal Selected Food Items: {selected_item}")
print(f"Total Cost: {total_cost:.2f} Baht")

if(total_cal >= min_cal and total_cal <= max_cal):
    print(f"\nTotal Cal: {total_cal} Kcal. ---- Calory Requirements: Satisfied :)")
else: print(f"\nTotal Cal: {total_cal} Kcal. ---- Calory Requirements: not Satisfied :(")
    
#Total Protein
if(total_protein >= Pconst):
    print(f"\nTotal Protein: {total_protein} g. ---- Protein Requirements: Satisfied :)")
else: print(f"\nTotal Protein: {total_protein} g. ---- Protein Requirements: Not Satisfied :(")

#Total Fat
if(total_fat <= Fconst):
    print(f"\nTotal fat: {total_fat} g. --------- Fat Requirements: Satisfied :)")
else: print(f"\nTotal fat: {total_fat} g. --------- Fat Requirements: Not Satisfied :(")

#Total Sugar
if(total_sugar <= Sconst):
    print(f"\nTotal sugar: {total_sugar} g. ------- Sugar Requirements: Satisfied :)")
else: print(f"\nTotal sugar: {total_sugar} g. ------- Sugar Requirements: Not Satisfied :(")    

#Total Sodium
if(total_sodium <= Sodium_const):
    print(f"\nTotal sodium: {total_sodium} mg. --- Sodium Requirements: Satisfied :)")
else: print(f"\nTotal sodium: {total_sodium} mg. --- Sodium Requirements: Not Satisfied :(")  

print(f"Computation Time: {execution_time:.2f} ")
print("\n")
